In [ ]:
import re, os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm_notebook
from glob import glob
from dapeng.models.unet import unet5
from dapeng.utils.keras_tools import ImageDirectoryIterator, x_tif_reader, predict_from_directory
% matplotlib inline

In [ ]:
model = unet5(256)
model.load_weights("ModelArchive/release_v1.0.hdf5")

In [ ]:
def imsave(savepath, array):
    Image.fromarray(array, mode="L").save(savepath)

In [ ]:
for row in tqdm_notebook(range(108694, 108924), desc=u"预测"):
    filepaths = glob("/Users/mac/Desktop/shouguang/{}/*.jpeg".format(row))
    gen = ImageDirectoryIterator([(filepath, None) for filepath in filepaths], x_tif_reader, None, batch_size=16,
                                 shuffle=False)
    predic_y = model.predict_generator(gen, steps=len(gen), verbose=0)
    predic_y_byte = (predic_y * 255).astype(np.uint8)
    for i, filepath in enumerate(filepaths):
        filename = os.path.basename(filepath)
        gray = predic_y_byte[i].reshape(256, 256)
        imsave(os.path.join("/Users/mac/Desktop/model_prediction", str(row)+"_"+os.path.splitext(filename)[0] + ".png"), gray)
